In [1]:
import sys
sys.path.append('../')
from utils import open_file
import glob
list_of_annotations = glob.glob('../../../jielin/msmo/annotation/*/*/*')


In [42]:
import re

def clean_text(text):
    # Remove non-alphanumeric characters
    cleaned_text = re.sub(r"[^\w\s]", "", text)
    
    # Remove extra whitespaces
    cleaned_text = re.sub(r"\s+", " ", cleaned_text)
    
    return cleaned_text

In [43]:
train_id = []
train_date = []
train_headline = []
train_article = []
train_abstract = []

dev_id = []
dev_date = []
dev_headline = []
dev_article = []
dev_abstract = []

test_id = []
test_date = []
test_headline = []
test_article = []
test_abstract = []


for idx in range(len(list_of_annotations)):
    instance = open_file(list_of_annotations[idx])
    key = instance['info']['video_id']
    title = instance['info']['title']
    
    if key[-4:] == '0021' or key[-4:] == '0022' or key[-4:] == '0023' \
        or key[-4:] == '0024' or key[-4:] == '0025':
            
        test_headline.append(title)
        test_id.append(key)
        test_date.append(0)
        test_article.append(clean_text(' '.join([item['summary'] for item in instance['transcript']])))
        test_abstract.append(clean_text(' '.join([item['summary'] for item in instance['summary']])))
        
    elif key[-4:] == '0026' or key[-4:] == '0027' or key[-4:] == '0028' \
        or key[-4:] == '0029':
        
        dev_headline.append(title)
        dev_id.append(key)
        dev_date.append(0)
        dev_article.append(clean_text(' '.join([item['summary'] for item in instance['transcript']])))
        dev_abstract.append(clean_text(' '.join([item['summary'] for item in instance['summary']])))
        
    else:
        
        train_headline.append(title)
        train_id.append(key)
        train_date.append(0)
        train_article.append(clean_text(' '.join([item['summary'] for item in instance['transcript']])))
        train_abstract.append(clean_text(' '.join([item['summary'] for item in instance['summary']])))
    

In [44]:
# Define column names
column_names = ["id", "date", "headline", "article", "abstract"]

# Open the file in write mode
with open('src/data/train_mms_joined.tsv', 'w') as tsv_file:
    # Write the header line
    header = '\t'.join(column_names) + '\n'
    tsv_file.write(header)
    
    # Iterate over the lists simultaneously using zip
    for item1, item2, item3, item4, item5 in zip(train_id, train_date, train_headline, train_article, train_abstract):
        # Write the values separated by tabs
        row = f"{item1}\t{item2}\t{item3}\t{item4}\t{item5}\n"
        tsv_file.write(row)
        
# Open the file in write mode
with open('src/data/test_mms_joined.tsv', 'w') as tsv_file:
    # Write the header line
    header = '\t'.join(column_names) + '\n'
    tsv_file.write(header)
    
    # Iterate over the lists simultaneously using zip
    for item1, item2, item3, item4, item5 in zip(test_id, test_date, test_headline, test_article, test_abstract):
        # Write the values separated by tabs
        row = f"{item1}\t{item2}\t{item3}\t{item4}\t{item5}\n"
        tsv_file.write(row)
        
# Open the file in write mode
with open('src/data/dev_mms_joined.tsv', 'w') as tsv_file:
    # Write the header line
    header = '\t'.join(column_names) + '\n'
    tsv_file.write(header)
    
    # Iterate over the lists simultaneously using zip
    for item1, item2, item3, item4, item5 in zip(dev_id, dev_date, dev_headline, dev_article, dev_abstract):
        # Write the values separated by tabs
        row = f"{item1}\t{item2}\t{item3}\t{item4}\t{item5}\n"
        tsv_file.write(row)

In [59]:

def time_to_seconds(time_string):
    hours, minutes, seconds = map(int, time_string.split(":"))
    total_seconds = hours * 3600 + minutes * 60 + seconds
    return total_seconds

In [61]:
def join_summaries_by_time(start_time, end_time, summaries):
    joined_summary = []
    for summary in summaries:
        if start_time <= time_to_seconds(summary['start_time']) <= end_time and start_time <= time_to_seconds(summary['end_time']) <= end_time:
            joined_summary.append(summary['summary'])
    return ' '.join(joined_summary)

In [62]:
train_id = []
train_date = []
train_headline = []
train_article = []
train_abstract = []

dev_id = []
dev_date = []
dev_headline = []
dev_article = []
dev_abstract = []

test_id = []
test_date = []
test_headline = []
test_article = []
test_abstract = []

count_0 = 0
for idx in range(len(list_of_annotations)):
    instance = open_file(list_of_annotations[idx])
    key = instance['info']['video_id']
    title = instance['info']['title']
    segments = instance['summary']
    transcripts = instance['transcript']
    
    
    count = 0
    for seg in segments:
        start_time = time_to_seconds(seg['start_time'])
        end_time = time_to_seconds(seg['end_time'])
                    
        seg_summary = join_summaries_by_time(start_time, end_time, transcripts)
    
        if key[-4:] == '0021' or key[-4:] == '0022' or key[-4:] == '0023' \
            or key[-4:] == '0024' or key[-4:] == '0025':
                
            test_headline.append(title)
            test_id.append(f'{key}_{count}')
            test_date.append(0)
            test_article.append(clean_text(seg_summary))
            test_abstract.append(clean_text(seg['summary']))
            
        elif key[-4:] == '0026' or key[-4:] == '0027' or key[-4:] == '0028' \
            or key[-4:] == '0029':
            
            dev_headline.append(title)
            dev_id.append(f'{key}_{count}')
            dev_date.append(0)
            dev_article.append(clean_text(seg_summary))
            dev_abstract.append(clean_text(seg['summary']))
            
        else:
            
            train_headline.append(title)
            train_id.append(f'{key}_{count}')
            train_date.append(0)
            train_article.append(clean_text(seg_summary))
            train_abstract.append(clean_text(seg['summary']))
    
    

In [63]:
# Define column names
column_names = ["id", "date", "headline", "article", "abstract"]

# Open the file in write mode
with open('src/data/train_mms_joined_2.tsv', 'w') as tsv_file:
    # Write the header line
    header = '\t'.join(column_names) + '\n'
    tsv_file.write(header)
    
    # Iterate over the lists simultaneously using zip
    for item1, item2, item3, item4, item5 in zip(train_id, train_date, train_headline, train_article, train_abstract):
        # Write the values separated by tabs
        row = f"{item1}\t{item2}\t{item3}\t{item4}\t{item5}\n"
        tsv_file.write(row)
        
# Open the file in write mode
with open('src/data/test_mms_joined_2.tsv', 'w') as tsv_file:
    # Write the header line
    header = '\t'.join(column_names) + '\n'
    tsv_file.write(header)
    
    # Iterate over the lists simultaneously using zip
    for item1, item2, item3, item4, item5 in zip(test_id, test_date, test_headline, test_article, test_abstract):
        # Write the values separated by tabs
        row = f"{item1}\t{item2}\t{item3}\t{item4}\t{item5}\n"
        tsv_file.write(row)
        
# Open the file in write mode
with open('src/data/dev_mms_joined_2.tsv', 'w') as tsv_file:
    # Write the header line
    header = '\t'.join(column_names) + '\n'
    tsv_file.write(header)
    
    # Iterate over the lists simultaneously using zip
    for item1, item2, item3, item4, item5 in zip(dev_id, dev_date, dev_headline, dev_article, dev_abstract):
        # Write the values separated by tabs
        row = f"{item1}\t{item2}\t{item3}\t{item4}\t{item5}\n"
        tsv_file.write(row)

In [13]:
import os
import numpy as np
thumbnail = 'src/data/thumbnails/'
keyframe = 'src/data/keyframes/'
video = 'src/data/videos/'

for i in sorted(os.listdir(thumbnail)):
    thumb_arr = np.load(os.path.join(thumbnail, i))
    key_arr = np.load(os.path.join(keyframe, i))
    video_arr = np.load(os.path.join(video, i))
    count = 0
    for j in range(key_arr.shape[0]):
        # np.save(f"src/data/thumbnails2/{i.split('.')[0]}_{count}.npy", thumb_arr)
        np.save(f"src/data/keyframes2/{i.split('.')[0]}_{count}.npy", key_arr[j].reshape((1, key_arr[j].shape[0])))
        # np.save(f"src/data/videos2/{i.split('.')[0]}_{count}.npy", video_arr)
    
        count+=1

In [74]:
import pandas as pd
import csv
df = pd.read_csv(
    'src/data/train_mms_joined_2.tsv',
    sep="\t",
    quoting=csv.QUOTE_NONE,
)

In [75]:
df = df.dropna()

In [76]:
df

,id,date,headline,article,abstract
0,COOPOA0009_0,0,How to Make Perfect POACHED EGGS - Cooking Basics,Hey everyone Its Natasha of NatashasKitchenco...,Intro
1,COOPOA0009_0,0,How to Make Perfect POACHED EGGS - Cooking Basics,you can poach one to four eggs at a time If yo...,What pot to use for poached eggs
2,COOPOA0009_0,0,How to Make Perfect POACHED EGGS - Cooking Basics,Im using large cold eggs right out of the refr...,Type of eggs to use
3,COOPOA0009_0,0,How to Make Perfect POACHED EGGS - Cooking Basics,before transferring it to the ramekin so you c...,Egg cracking tip
4,COOPOA0009_0,0,How to Make Perfect POACHED EGGS - Cooking Basics,Once you have all of your eggs cracked into in...,Preparing the water
...,...,...,...,...,...
27486,SPOFOO0005_0,0,Behind The Crest: An Inside Look At USMNT Matc...,were about to do a broadcast walkthrough with ...,Broadcast Walkthrough
27487,SPOFOO0005_0,0,Behind The Crest: An Inside Look At USMNT Matc...,next up so item 44 similar to what we did in i...,PreGame Light Show
27488,SPOFOO0005_0,0,Behind The Crest: An Inside Look At USMNT Matc...,for world cup qualifiers theres a theres a big...,Security
27489,SPOFOO0005_0,0,Behind The Crest: An Inside Look At USMNT Matc...,a production in orlando is not going to be the...,Walkouts


In [77]:
df.to_csv('src/data/train_mms_joined_2.tsv', sep='\t', index=False)

In [58]:
import math
for i in df.article:
    if i == 'nan':
        print('s')

In [70]:
value_to_delete = 'Simple, Easy & it was DELICIOUS!'
df = df.drop(df[df['id'] == value_to_delete].index)

In [71]:
df.to_csv('src/data/test_mms_2.tsv', sep='\t', index=False)

In [34]:
len(df)

850

In [49]:
for i in range(len(df)):
   if df.iloc[i, :][0] == 'Simple, Easy & it was DELICIOUS!':
       print('hello')
   

In [23]:
for i in range(len(df)):
   if df.iloc[i, :][0] == 'Simple, Easy & it was DELICIOUS!':
       print(df.iloc[i,:])

id                           Simple, Easy & it was DELICIOUS!
date        ["hi everyone today I'm going to do my", 'simp...
headline                                                Intro
article                                                   NaN
abstract                                                  NaN
Name: 73, dtype: object
id                           Simple, Easy & it was DELICIOUS!
date        ['first I am going to add a couple of', 'table...
headline                                              Cooking
article                                                   NaN
abstract                                                  NaN
Name: 75, dtype: object
